In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
sys.path.append("../")
import folderparser as fp
import folderscraper as fs
from interfacePlots import *
from config import cfg

In [8]:
EFOLDERS = [os.path.join(cfg.path.c, 'HBHByielded', s) for s in ['LapRD']]
EXPORTFOLDER = os.path.join(cfg.path.fig, 'yieldingsweep', 'plots')

# Generating files

In [10]:
import pandas as pd
o = []
for topfolder in EFOLDERS:
    for f in os.listdir(topfolder):
        ffull = os.path.join(topfolder, f)
        try:
            fs.populate(ffull)
        except:
            pass
        else:
            l = fp.legendUnique(ffull)
            if len(l)>0:
                o.append(l)
p = pd.DataFrame(o)
p.to_csv(os.path.join(cfg.path.fig, 'yieldingsweep','LapRD_legend_general.csv'))

In [ ]:
for topfolder in EFOLDERS:
    for folder in fp.caseFolders(topfolder):
        intm.sumAndSteady(folder, True)

# Plots

`('nuink', 'tau0ink', 'kink', 'nink', 'nusup', 'tau0sup', 'ksup', 'nsup', 'sigma')`

In [ ]:
txtPlots0(EFOLDERS[0], EXPORTFOLDER, xvar='kink*ksup', yvar='ksup/kink*nink', split=True, overwrite=True)

In [ ]:
for i, TOPFOLDER in enumerate(EFOLDERS):
    xvar = os.path.basename(TOPFOLDER)+'ink'
    yvar = os.path.basename(TOPFOLDER)+'sup'
    txtPlots0(TOPFOLDER, EXPORTFOLDER, xvar=xvar, yvar=yvar, overwrite=False)
    timePlots(TOPFOLDER, EXPORTFOLDER, xvar=xvar, yvar=yvar, overwrite=True)
    runtimePlots0(TOPFOLDER, EXPORTFOLDER, xvar=xvar, yvar=yvar, overwrite=True)
        
    xposition = 5
    for t in [2.5]:
        for s in [['arean', 0.7, 1.8], ['vertdispn', 0, 1], ['aspectratio', 0.6, 1.7], ['speeddecay', 0.5, 1.0]]:
            metricPlots(TOPFOLDER, EXPORTFOLDER, t, xposition, s[0], xvar=xvar, yvar=yvar, split=True, sigmalist=[0,40], overwrite=True, tmin=s[1], tmax=s[2])
            
    for t in [1, 2.5]:
        for sigma in [0, 40]:
            for ta in ['y_umag', 'y_viscy', 'y_uslicey', 'x_uslicex', 'x_viscx']:
                picPlots0(TOPFOLDER, EXPORTFOLDER, t, sigma, tag=ta, xvar=xvar, yvar=yvar, overwrite=True)
    
    for t in [2.5]:
        xposition = 5
        XSPlots0(TOPFOLDER, EXPORTFOLDER, t, xposition, [0, 40], xvar=xvar, yvar=yvar, overwrite=True)
    
    for sigma in [0, 40]:
        imsize=1.3
        steadyPlots(TOPFOLDER, imsize, EXPORTFOLDER, [sigma], xvar=xvar, yvar=yvar, overwrite=True)

In [ ]:
import pandas as pd
metrics = []
for TOPFOLDER in EFOLDERS:
    for f in os.listdir(TOPFOLDER):
        try:
            m = metricVals(os.path.join(TOPFOLDER, f), 2.5, 5, ['arean', 'vertdispn', 'aspectratio', 'speeddecay'])
        except:
            pass
        else:
            metrics.append(m)
df = pd.DataFrame(metrics)
df

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3,sharex='col', sharey='row',figsize=(6.5,6.5))
cmap = plt.get_cmap('tab20c')

for t in range(len(EFOLDERS)):
    TOPFOLDER = EFOLDERS[t]
    metrics = []
    for f in os.listdir(TOPFOLDER):
        try:
            m = metricVals(os.path.join(TOPFOLDER, f), 2.5, 5, ['arean', 'vertdispn', 'aspectratio', 'speeddecay'])
        except:
            pass
        else:
            metrics.append(m)
    df = pd.DataFrame(metrics)

    df2 = df[df['arean']<3]
    df2 = df2[df2['vertdispn']>-0.1]
    df2 = df2[df2['vertdispn']<1.2]
    df2 = df2[df2['speeddecay']<1.1]

    slist = df.keys()
    for i in range(4):
        for j in range(i):
            color = cmap(t/3)
            if j==0 and i==1:
                axs[j,i-1].scatter(df2[slist[i]], df2[slist[j]], s=5, color=color, label=os.path.basename(TOPFOLDER))
            else:
                axs[j,i-1].scatter(df2[slist[i]], df2[slist[j]], s=5, color=color)
            axs[j,i-1].set_xlabel(slist[i])
            axs[j,i-1].set_ylabel(slist[j])
axs[0,0].legend()
